# HW — 기계공학 응용 예제

Practice 06에서 학습한 4가지 선형 모델을 **기계공학 분야의 실제 데이터**로 적용합니다.

| # | Model | Dataset | Task |
|---|---|---|---|
| 1 | Linear Regression | Auto MPG (UCI) | 차량 무게 → 연비 예측 |
| 2 | Perceptron | Steel Plates Faults (UCI) | 결함 유형 이진 분류 |
| 3 | Logistic (binary) | Steel Plates Faults (동일) | 결함 확률 추정 |
| 4 | Logistic (multi) | Steel Plates Faults (동일) | 결함 유형 3종 분류 |

> **Note:** 실제 데이터는 sklearn 내장 데이터와 달리 스케일이 다양하므로,  
> **표준화(standardization)** 과정이 추가됩니다: $x' = \frac{x - \mu}{\sigma}$

In [1]:
# UCI ML Repository 데이터 로딩 패키지 (첫 실행 시 설치 필요)
%pip install -q ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['axes.unicode_minus'] = False

---
# 1. Linear Regression — 자동차 연비 예측

**Auto MPG Dataset** ([UCI #9](https://archive.ics.uci.edu/dataset/9/auto+mpg))를 사용하여  
차량의 **무게(weight)**로 **연비(mpg)**를 예측합니다.

> **기계공학과의 관련성:**  
> 자동차 무게는 엔진 설계, 차체 재료 선택, 연비 규제 등 기계공학의 핵심 변수입니다.  
> 무거운 차량 → 높은 구동 저항 → 낮은 연비라는 직관적인 물리 관계가 있습니다.

$$\hat{y} = \mathbf{Xw}, \qquad J = \|\mathbf{Xw} - \mathbf{y}\|^2, \qquad \frac{\partial J}{\partial \mathbf{w}} = 2\mathbf{X}^T(\mathbf{Xw} - \mathbf{y})$$

### 데이터 로드 + 전처리

In [3]:
# Auto MPG: 차량 특성으로 연비(mpg) 예측
auto_mpg = fetch_ucirepo(id=9)

# weight(차량 무게) 1개 feature만 사용
x_raw = auto_mpg.data.features['weight'].values.astype(float).reshape(-1, 1)
y = auto_mpg.data.targets['mpg'].values.astype(float).reshape(-1, 1)

# 결측치(NaN) 제거 — 실제 데이터에는 빈 값이 있을 수 있음
print(f"결측치 제거 전: {len(x_raw)}개")
nan_x = np.isnan(x_raw).flatten()    # x에서 NaN인 행 찾기
nan_y = np.isnan(y).flatten()        # y에서 NaN인 행 찾기
valid = ~nan_x & ~nan_y              # 둘 다 NaN이 아닌 행만 선택
x_raw = x_raw[valid]
y = y[valid]
print(f"결측치 제거 후: {len(x_raw)}개 (제거: {(~valid).sum()}개)")

# 표준화: gradient descent 수렴을 위해 feature 스케일 통일
x_mean, x_std = x_raw.mean(), x_raw.std()
x = (x_raw - x_mean) / x_std

# Augmented matrix: [1, x]
X = np.hstack([np.ones((len(x), 1)), x])

print(f"\nX shape: {X.shape}, y shape: {y.shape}")
print(f"Weight: {x_raw.min():.0f} ~ {x_raw.max():.0f} lbs")
print(f"MPG:    {y.min():.1f} ~ {y.max():.1f}")

결측치 제거 전: 398개
결측치 제거 후: 398개 (제거: 0개)

X shape: (398, 2), y shape: (398, 1)
Weight: 1613 ~ 5140 lbs
MPG:    9.0 ~ 46.6


In [4]:
# 데이터 미리보기
pd.DataFrame({'weight (lbs)': x_raw.flatten(), 'mpg': y.flatten()}).head(10)

,weight (lbs),mpg
0,3504.0,18.0
1,3693.0,15.0
2,3436.0,18.0
3,3433.0,16.0
4,3449.0,17.0
5,4341.0,15.0
6,4354.0,14.0
7,4312.0,14.0
8,4425.0,14.0
9,3850.0,15.0


### 학습

In [5]:
# 학습 코드를 작성하시오.
# - w = [w0, w1], alpha = 0.001 (학습률), n_iter = 3000 (반복 횟수) 설정
# - 매 iteration의 MSE loss를 loss_hist에 기록하시오.



### 평가 + 시각화

In [ ]:
# 그래프를 subplot(1,2)로 그리고, R² 를 계산하여 보고하시오.
# - 왼쪽: scatter plot과 회귀선 (x축: Weight, y축: MPG)
# - 오른쪽: epoch별 MSE Loss

# R² (결정 계수): 1에 가까울수록 좋은 모델
# y_pred = X @ w
# ss_res = np.sum((y - y_pred)**2)
# ss_tot = np.sum((y - y.mean())**2)
# r2 = 1 - ss_res / ss_tot
# print(f"R² = {r2:.4f}")


---
# Steel Plates Faults — 공유 데이터 로드

**Steel Plates Faults Dataset** ([UCI #198](https://archive.ics.uci.edu/dataset/198/steel+plates+faults))는  
철강 제조 공정에서 발생하는 **7가지 표면 결함**을 27개 측정값으로 분류하는 데이터셋입니다.

> **기계공학과의 관련성:**  
> 철강 제조 공정의 **자동 품질 검사**는 스마트 제조의 핵심입니다.  
> 결함 유형을 자동으로 분류하면 공정 개선 방향을 파악할 수 있습니다.

Section 2–4에서 이 데이터를 **이진(2-class) → 다중(3-class)**으로 확장하며 사용합니다.

In [7]:
# Steel Plates Faults 데이터 로드 (Section 2, 3, 4 공유)
steel = fetch_ucirepo(id=198)
X_steel = steel.data.features
y_steel = steel.data.targets

# 결함 유형별 샘플 수
all_labels = y_steel.values.argmax(axis=1)
fault_names = y_steel.columns.tolist()
print("결함 유형 분포:")
for i, name in enumerate(fault_names):
    print(f"  {i}: {name:15s} ({(all_labels==i).sum()}개)")

결함 유형 분포:
  0: Pastry          (158개)
  1: Z_Scratch       (190개)
  2: K_Scratch       (391개)
  3: Stains          (72개)
  4: Dirtiness       (55개)
  5: Bumps           (402개)
  6: Other_Faults    (673개)


In [8]:
# 데이터 미리보기 (features + 결함 유형)
preview = X_steel.head(10).copy()
preview['fault'] = [fault_names[l] for l in all_labels[:10]]
preview

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,...,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Minimum_of_Luminosity,fault
0,42,50,270900,270944,267,17,44,24220,108,1687,...,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,76,Pastry
1,645,651,2538079,2538108,108,10,30,11397,123,1687,...,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,84,Pastry
2,829,835,1553913,1553931,71,8,19,7972,125,1623,...,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,99,Pastry
3,853,860,369370,369415,176,13,45,18996,126,1353,...,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,99,Pastry
4,1289,1306,498078,498335,2409,60,260,246930,126,1353,...,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,37,Pastry
5,430,441,100250,100337,630,20,87,62357,127,1387,...,1.0000,1.0,2.7993,1.0414,1.9395,0.8736,-0.2267,0.9874,64,Pastry
6,413,446,138468,138883,9052,230,432,1481991,199,1687,...,0.9607,1.0,3.9567,1.5185,2.6181,0.9205,0.2791,1.0000,23,Pastry
7,190,200,210936,210956,132,11,20,20007,172,1687,...,1.0000,1.0,2.1206,1.0000,1.3010,0.5000,0.1841,0.3359,124,Pastry
8,330,343,429227,429253,264,15,26,29748,148,1687,...,1.0000,1.0,2.4216,1.1139,1.4150,0.5000,-0.1197,0.5593,53,Pastry
9,74,90,779144,779308,1506,46,167,180215,143,1687,...,0.9820,1.0,3.1778,1.2041,2.2148,0.9024,-0.0651,1.0000,53,Pastry


---
# 2. Perceptron — 철판 결함 이진 분류

**K_Scratch**(긁힘)와 **Bumps**(돌출) 두 가지 결함을  
**결함 면적**과 **철판 두께** 두 변수만으로 분류합니다.

$$\sigma(z) = \begin{cases} 1 & z \geq 0 \\ 0 & z < 0 \end{cases}, \qquad \mathbf{w} \leftarrow \mathbf{w} + \rho\, \mathbf{X}^T(\mathbf{y} - \sigma(\mathbf{z}))$$

### 데이터 준비 (2-class, 2-feature)

In [9]:
# K_Scratch(idx=2) vs Bumps(idx=5) — 이진 분류
mask_bin = np.isin(all_labels, [2, 5])
y_bin = (all_labels[mask_bin] == 5).astype(int)   # K_Scratch=0, Bumps=1

# 2 features: 결함 면적(로그), 철판 두께
f1_raw = X_steel.loc[mask_bin, 'LogOfAreas'].values.astype(float)
f2_raw = X_steel.loc[mask_bin, 'Steel_Plate_Thickness'].values.astype(float)

# 표준화
f1_mean, f1_std = f1_raw.mean(), f1_raw.std()
f2_mean, f2_std = f2_raw.mean(), f2_raw.std()
f1_norm = (f1_raw - f1_mean) / f1_std
f2_norm = (f2_raw - f2_mean) / f2_std

# Augmented matrix: [1, f1, f2]
X_bin = np.column_stack([np.ones(len(y_bin)), f1_norm, f2_norm])

print(f"X shape: {X_bin.shape}")
print(f"K_Scratch (y=0): {(y_bin==0).sum()}, Bumps (y=1): {(y_bin==1).sum()}")

X shape: (793, 3)
K_Scratch (y=0): 391, Bumps (y=1): 402


In [10]:
# 데이터 미리보기
label_map = {0: 'K_Scratch', 1: 'Bumps'}
pd.DataFrame({
    'LogOfAreas': f1_raw, 'Steel_Plate_Thickness': f2_raw,
    'y': y_bin, 'class': [label_map[v] for v in y_bin]
}).head(10)

,LogOfAreas,Steel_Plate_Thickness,y,class
0,1.8573,40.0,0,K_Scratch
1,1.8195,40.0,0,K_Scratch
2,1.7853,40.0,0,K_Scratch
3,1.8751,40.0,0,K_Scratch
4,2.6464,40.0,0,K_Scratch
5,1.8808,40.0,0,K_Scratch
6,2.1761,40.0,0,K_Scratch
7,2.9504,40.0,0,K_Scratch
8,1.8692,40.0,0,K_Scratch
9,2.1335,40.0,0,K_Scratch


### 학습

In [11]:
# 학습 코드를 작성하시오.
# - step 함수: z >= 0 이면 1, 아니면 0
# - 매 epoch의 Accuracy를 acc_hist에 기록하시오.



### 평가 + 시각화

In [12]:
# 그래프를 subplot(1,2)로 그리시오.
# - 왼쪽: feature scatter plot (LogOfAreas vs Thickness)과 결정 경계
# - 오른쪽: epoch별 Accuracy (%)



---
# 3. Logistic Regression (Binary) — 결함 확률 추정

Section 2와 동일한 데이터를 사용하지만, Perceptron의 이진 출력(0 or 1) 대신  
Sigmoid 함수로 **결함이 Bumps일 확률**을 추정합니다.

$$\sigma(z) = \frac{1}{1+e^{-z}}, \qquad J = -\sum_n \bigl[ y_n \log \sigma(z_n) + (1-y_n) \log(1-\sigma(z_n)) \bigr]$$

$$\frac{\partial J}{\partial \mathbf{w}} = \mathbf{X}^T(\sigma(\mathbf{z}) - \mathbf{y}), \qquad \mathbf{w} \leftarrow \mathbf{w} - \rho \frac{\partial J}{\partial \mathbf{w}}$$

### 학습

In [13]:
# 학습 코드를 작성하시오.
# - sigmoid 함수: 1 / (1 + exp(-z))
# - 매 epoch의 Accuracy를 acc_hist에 기록하시오.



### 평가 + 시각화

In [14]:
# 그래프를 subplot(1,2)로 그리시오.
# - 왼쪽: feature scatter plot (LogOfAreas vs Thickness)과 결정 경계
# - 오른쪽: epoch별 Accuracy (%)



---
# 4. Logistic Regression (Multi-class) — 결함 유형 3종 분류

동일한 Steel Plates 데이터에서 **3가지 결함 유형**을 4개 feature로 분류합니다.

$$o_k = \frac{e^{z_k}}{\sum_q e^{z_q}}, \qquad J = -\sum_n \log o_{n,\, y_n}$$

$$\frac{\partial J}{\partial \mathbf{W}} = (\mathbf{O} - \mathbf{Y})^T \mathbf{X}, \qquad \mathbf{W} \leftarrow \mathbf{W} - \rho \frac{\partial J}{\partial \mathbf{W}}$$

### 데이터 준비 (3-class, 4-feature)

Section 2의 2종(K_Scratch, Bumps)에 **Z_Scratch**(190개)를 추가하고, feature를 4개로 확장합니다.

| Feature | 설명 | 범위 |
|---|---|---|
| **LogOfAreas** | 결함 영역 면적의 로그값 — 결함의 **크기** | 0.3 ~ 5.2 |
| **Steel_Plate_Thickness** | 철판 두께 (mm) — **재료 특성** | 40 ~ 300 |
| **Edges_Index** | 결함 경계의 선명도 — 경계가 뚜렷할수록 높음 | 0 ~ 1 |
| **Luminosity_Index** | 결함 부위의 밝기 — 결함의 **시각적 특성** | -1 ~ 0.6 |

In [15]:
# 3개 결함 유형: K_Scratch(391), Bumps(402), Z_Scratch(190)
target_classes = {2: 0, 5: 1, 1: 2}   # K_Scratch→0, Bumps→1, Z_Scratch→2
target_names = ['K_Scratch', 'Bumps', 'Z_Scratch']

mask_mc = np.isin(all_labels, list(target_classes.keys()))
y_mc = np.array([target_classes[l] for l in all_labels[mask_mc]])

# 4개 feature
feature_cols = ['LogOfAreas', 'Steel_Plate_Thickness', 'Edges_Index', 'Luminosity_Index']
X_raw_mc = X_steel.loc[mask_mc, feature_cols].values.astype(float)

N, C, d = len(y_mc), 3, 4

# 표준화
X_mc_mean = X_raw_mc.mean(axis=0)
X_mc_std  = X_raw_mc.std(axis=0)
X_mc_norm = (X_raw_mc - X_mc_mean) / X_mc_std

# Augmented matrix
X_mc = np.column_stack([np.ones(N), X_mc_norm])

# One-hot encoding
Y_oh = np.zeros((N, C))
Y_oh[np.arange(N), y_mc] = 1

print(f"X shape: {X_mc.shape}, N: {N}")
for i, name in enumerate(target_names):
    print(f"  Class {i} ({name}): {(y_mc==i).sum()}")

X shape: (983, 5), N: 983
  Class 0 (K_Scratch): 391
  Class 1 (Bumps): 402
  Class 2 (Z_Scratch): 190


In [16]:
# 데이터 미리보기
pd.DataFrame(
    X_raw_mc, columns=feature_cols
).assign(y=y_mc, **{'class': [target_names[v] for v in y_mc]}).head(10)

,LogOfAreas,Steel_Plate_Thickness,Edges_Index,Luminosity_Index,y,class
0,2.0899,100.0,0.6124,0.0072,2,Z_Scratch
1,1.7482,70.0,0.0103,-0.0748,2,Z_Scratch
2,2.3541,70.0,0.0547,-0.1163,2,Z_Scratch
3,2.6395,70.0,0.0576,-0.3004,2,Z_Scratch
4,1.9243,70.0,0.2851,-0.1485,2,Z_Scratch
5,1.7993,70.0,0.2142,-0.0387,2,Z_Scratch
6,3.2014,70.0,0.0576,-0.2431,2,Z_Scratch
7,3.3554,70.0,0.2083,-0.1426,2,Z_Scratch
8,2.7168,70.0,0.1004,-0.0564,2,Z_Scratch
9,2.5966,70.0,0.1195,-0.1797,2,Z_Scratch


### 학습

In [17]:
# 학습 코드를 작성하시오.
# - softmax 함수: exp(z) / sum(exp(z))
# - 매 epoch의 Accuracy를 acc_hist에 기록하시오.



### 평가 + 시각화

In [18]:
# 그래프를 subplot(1,2)로 그리시오.
# - 왼쪽: epoch별 CE Loss
# - 오른쪽: epoch별 Accuracy (%)



---
# 5. 비교 요약

| | Linear Regression | Perceptron | Logistic (Binary) | Logistic (Multi) |
|---|---|---|---|---|
| **Dataset** | Auto MPG | Steel Plates Faults | Steel Plates (동일) | Steel Plates (동일) |
| **Task** | 무게 → 연비 | K_Scratch vs Bumps | 결함 확률 | 3종 결함 분류 |
| **Features** | weight (1개) | LogOfAreas, Thickness (2개) | 동일 (2개) | 4개 |
| **Activation** | identity | step | $\sigma(z)=\frac{1}{1+e^{-z}}$ | softmax |
| **Loss** | $\|\mathbf{Xw}-\mathbf{y}\|^2$ | miscount | $-\sum y\log\sigma$ | $-\sum \log o_{y_n}$ |
| **Metric** | R² | Accuracy | Accuracy | Accuracy |

> 학습 코드는 Practice 06과 **완전히 동일**합니다. 데이터만 기계공학 문제로 교체했습니다.